<a href="https://colab.research.google.com/github/krDaria/dl-course/blob/master/nmt_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Open With Colab](https://colab.research.google.com/github/m12sl/dl_cshse_2019/blob/master/seminars/x2seq/nmt.ipynb)

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from collections import Counter

import unicodedata
import re
import string

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

from tqdm import tqdm_notebook as tqdm

In [2]:
! wget https://github.com/m12sl/dl_cshse_2019/raw/master/seminars/x2seq/eng-rus.tar.gz
! tar xzvf eng-rus.tar.gz

--2019-06-11 17:37:01--  https://github.com/m12sl/dl_cshse_2019/raw/master/seminars/x2seq/eng-rus.tar.gz
Resolving github.com (github.com)... 13.250.177.223
Connecting to github.com (github.com)|13.250.177.223|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/m12sl/dl_cshse_2019/master/seminars/x2seq/eng-rus.tar.gz [following]
--2019-06-11 17:37:02--  https://raw.githubusercontent.com/m12sl/dl_cshse_2019/master/seminars/x2seq/eng-rus.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7020408 (6.7M) [application/octet-stream]
Saving to: ‘eng-rus.tar.gz.2’

eng-rus.tar.gz.2    100%[===================>]   6.69M  --.-KB/s    in 0.07s   

2019-06-11 17:37:03 (92.3 MB/s) - ‘eng-rus.tar.gz.2’ saved [702

## Наивный вариант представления текстов

0. Нормализовать написание
1. Отфильтруем все спецсимволы
2. Разобьем по пробелам, сделаем *наивную токенизацию*

In [3]:
# Приготовим данные и посмотрим на них
# Кроме словаря нас интересует еще набор символов
raw_alphabet = set()
alphabet = set()
def normalize(s):
    return "".join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')


def preprocess(s):
    raw_alphabet.update(s)
    s = normalize(s.lower().strip())
    s = re.sub(r"[^a-zа-я?.,!]+", " ", s)
    s = re.sub(r"([.!?])", r" \1", s)
    alphabet.update(s)
    return s

pairs = []
with open('eng-rus.txt', 'r') as fin:
    for line in tqdm(fin.readlines()):
        pair = [preprocess(_) for _ in line.split('\t')]
        pairs.append(pair)
        
print("RAW alphabet {} symbols:".format(len(raw_alphabet)), 
      "".join(sorted(raw_alphabet)))
print("After preprocessing {} symbols: ".format(len(alphabet)), 
      "".join(sorted(alphabet)))
print("There are {} pairs".format(len(pairs)))
print(pairs[10000])


RAW alphabet 174 symbols: 
 !"$%&'()+,-./0123456789:;?@ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz «°º»ãçéêîïóöúǘЁАБВГДЕЖЗИЙКЛМНОПРСТУФХЦЧШЩЬЭЮЯабвгдежзийклмнопрстуфхцчшщъыьэюяёׁ​–—―‘’… ‽₂€№
After preprocessing 62 symbols:   !,.?abcdefghijklmnopqrstuvwxyzабвгдежзиклмнопрстуфхцчшщъыьэюя
There are 336666 pairs
['it s too fast .', 'это слишком быстро .']


Каждому слову поставим в соответсвие номер + нам потребуются спецтокены для начала и конца последовательности и для неизвестных слов.
`<SOS>, <EOS>, <UNK>`

У нас два языка, для работы с каждым нам потребуются словами и функции для перевода из слов в номера и обратно.





In [4]:

COMMON_TOKENS = ['PAD', 'SOS', 'EOS', 'UNK']


def build_vocabs(sents, max_size=1000):
    cnt = Counter()
    for s in sents:
        cnt.update(s.split(' '))
        
    print('There are {} tokens'.format(len(cnt)))
    toks = COMMON_TOKENS + [_[0] for _ in cnt.most_common(max_size - len(COMMON_TOKENS))]
    tok2idx = {t: i for i, t in enumerate(toks)}
    idx2tok = {i: t for t, i in tok2idx.items()}
    print('Truncate to {} toks'.format(len(tok2idx)))
    return tok2idx, idx2tok


eng, rus = list(zip(*pairs))
rus2idx, idx2rus = build_vocabs(rus, max_size=10000)
eng2idx, idx2eng = build_vocabs(eng, max_size=5000)

def sentence2idx(s, tok2idx):
    tokens = preprocess(s).split(' ')
    unk = tok2idx['UNK']
    return [tok2idx['SOS']] + [tok2idx.get(_, unk) for _ in tokens] + [tok2idx['EOS']]


def idx2sentence(s, idx2tok):
    return " ".join(idx2tok[_] for _ in s)

There are 57309 tokens
Truncate to 10000 toks
There are 17459 tokens
Truncate to 5000 toks


In [5]:
# проверим консистентность преобразований
x = sentence2idx('Привет мир!', rus2idx)
print(x)
print(idx2sentence(x, idx2rus))

x = sentence2idx('Hello world!', eng2idx)
print(x)
print(idx2sentence(x, idx2eng))

[1, 2540, 1265, 83, 2]
SOS привет мир ! EOS
[1, 1961, 440, 175, 2]
SOS hello world ! EOS


## Работа с последовательностями произвольной длинны в pytorch

Нам нужно уметь генерировать батчи тензоров `[bs, 1, seq_len]`.
Но в нашем датасете семплы разной длины:

- мы могли бы подрезать все до минимальной
- паддить до максимальной
- выбрать какую-то среднюю длину

In [0]:
# сделаем датасет с закодированными парами:
class EngRusDataset(Dataset):
    def __init__(self, pairs):
        self.pairs = pairs
    
    def __len__(self):
        return len(self.pairs)
    
    def __getitem__(self, item):
        eng, rus = self.pairs[item]
        return dict(
            eng=eng,
            rus=rus,
        )

encoded = []
for eng, rus in tqdm(pairs):
    a = sentence2idx(eng, eng2idx)
    b = sentence2idx(rus, rus2idx)
    encoded.append((a, b))
    
ds = EngRusDataset(encoded)

Давайте соберем наивный DataLoader и посмотрим как он делает батчи:


In [0]:
dataloader = DataLoader(ds, batch_size=8, shuffle=True)
it = iter(dataloader)

In [0]:
next(it)['eng']

В моем случае, результат запуска был таков:
```
[tensor([1, 1, 1, 1, 1, 1, 1, 1]),
 tensor([ 6,  7,  6, 15,  5,  6,  5, 62]),
 tensor([ 48,  34,  83,   7,  32, 221,  22,  43]),
 tensor([  5, 143,  37,  36, 129,  12,  11,  66]),
 tensor([  73, 1258,  279,    8,    6,  555,   41,   10]),
 tensor([  8, 140,   8, 628,  20,  96,  13, 270]),
 tensor([  47,    4,   15,   18,   55,  269,    6, 1287]),
 tensor([ 58,   2,  13, 140, 193, 140, 171, 140])]
```

Какие странности здесь видны?
1. Это не тензор, а список тензоров
2. На `<EOS>` (2) оканчивается только один пример, остальные подрезаны под его длину.

Мы бы хотели западдить все примеры до длины максимального в батче. 
Но на этапе подготовки семпла мы не знаем соседей по батчу!
Нам пригодиться параметр collate_fn в конструкторе DataLoader:

```
def collate_fn(samples):
    # samples -- список семплов-словарей
    <...>
    return batch
```

In [0]:
def collate_fn(samples):
    PAD = 0
    def _pad_to_longest(lst, pad_left=False):
        longest = max(len(s) for s in lst)
        if pad_left:
            return torch.LongTensor([[PAD] * (longest - len(s)) + s for s in lst])
        else:
            return torch.LongTensor([s + [PAD] * (longest - len(s)) for s in lst])
        
    eng = [s['eng'] for s in samples]
    rus = [s['rus'] for s in samples]
    
    return dict(
        rus=_pad_to_longest(rus, pad_left=False),
        eng=_pad_to_longest(eng, pad_left=True),
    )

dataloader = DataLoader(ds, batch_size=8, shuffle=True, collate_fn=collate_fn)
it = iter(dataloader)
next(it)['eng']

In [0]:
# Теперь напишем модельку, которая может переводить!
# Соберем модель из двух частей:
# - Encoder на RNN
# - Decoder на RNN

class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, vocab_size, layers=1):
        super().__init__()
        self.layers = layers
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size
        
        self.embeddings = nn.Embedding(vocab_size, hidden_size)
        self.rnn = nn.GRU(hidden_size, hidden_size, batch_first=True, num_layers=layers)
        
    def forward(self, input, hidden):
        embedded = self.embeddings(input)
        output, hidden = self.rnn(embedded, hidden)
        return output, hidden
    
    def init_hidden(self, batch_size=1, device=None):
        # be aware about dimension! https://pytorch.org/docs/stable/nn.html#torch.nn.GRU
        return torch.zeros(self.layers, batch_size, self.hidden_size, device=device)


enc = EncoderRNN(256, len(eng2idx))
x = next(it)['eng']
print(x.shape)
hidden = enc.init_hidden(8)
out, hidden = enc(x, hidden)
print(out.shape, hidden.shape)

In [0]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, vocab_size, layers=1):
        super().__init__()
        self.layers = layers
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size
        
        self.embeddings = nn.Embedding(vocab_size, hidden_size)
        self.rnn = nn.GRU(hidden_size, hidden_size, batch_first=True, num_layers=layers)
        self.out = nn.Linear(hidden_size, vocab_size)
        self.softmax = nn.LogSoftmax(dim=2)
        
    def forward(self, input, hidden):
        embedded = self.embeddings(input)
        output, hidden = self.rnn(embedded, hidden)
        output = self.softmax(self.out(output))
        return output, hidden
        
    def init_hidden(self, batch_size=1, device=None):
        return torch.zeros(self.layers, batch_size, self.hidden_size, device=device)
    
# декодер получит тензор с закодированным состоянием и батч первых токенов последовательности для генерации

In [0]:
y = next(it)['rus']
dec = DecoderRNN(256, len(rus2idx))

In [0]:
# проверяем размерности подаваемых и возвращаемых тензоров на каждом шаге
for i in range(0, y.shape[1]):
    t = y[:, i].view(-1, 1)
    print(t.shape)
    o, z = dec(t, hidden)
    print(o.shape, z.shape)

In [0]:
device = "cuda"
encoder = EncoderRNN(256, len(eng2idx)).to(device)
decoder = DecoderRNN(256, len(rus2idx)).to(device)
optimizer = torch.optim.Adam(list(encoder.parameters()) + list(decoder.parameters()), lr=1e-2)
dataloader = DataLoader(ds, batch_size=128, shuffle=True, collate_fn=collate_fn)

log = []
def train_model(dataloader, optimizer, teacher_forcing=True):
    encoder.train()
    decoder.train()
    
    for batch in tqdm(dataloader):
        eng = batch['eng'].to(device)
        rus = batch['rus'].to(device)
        encoder_hidden = encoder.init_hidden(eng.size(0)).to(device)
        encoder_outputs, hidden = encoder(eng, encoder_hidden)
        
        # мы добавляли <SOS> во все последовательности, так что предсказывать будем начиная со второй позиции
        loss = 0.0
        weight = 0.0
        x = rus[:, 0].view(-1, 1)
        for i in range(1, rus.size(1)):
            out, hidden = decoder(x, hidden)
            
            target = rus[:, i].view(-1, 1)
            # маскируем все паддинги
            mask = 1.0 * (target > 0)
            weight += 1.0 * mask.sum()
            loss += torch.sum(F.nll_loss(out.squeeze(1), target.squeeze(1)) * mask.float())
            
            if teacher_forcing:
                x = rus[:, i].view(-1, 1)
            else:
                # здесь могло бы быть семплирование из вероятностей
                _, topi = out.topk(1, dim=-1)
                x = topi.squeeze(-1).detach()
        loss /= (weight.float() + 1e-5)
                
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        log.append(loss.item())

train_model(dataloader, optimizer, teacher_forcing=True)

In [0]:
plt.plot(log)

In [0]:
encoder.eval()
decoder.eval()
encoder = encoder.to("cpu")
decoder = decoder.to("cpu")

def evaluate(sentence, T=1.0):
    encoded = sentence2idx(sentence, eng2idx)
    output = []
    print(encoded)
    bs = 10
    with torch.no_grad():
      
        z = torch.LongTensor(encoded).view(1, -1).repeat(bs, 1)
        encoder_outputs, hidden = encoder(z, encoder.init_hidden(bs))
        x = torch.LongTensor([1]).view(1, 1).repeat(bs, 1)
        for i in range(20):
            out, hidden = decoder(x, hidden)
            x = torch.multinomial(F.softmax(out / T, dim=-1).squeeze(1), 1)
#             _, topi = out.topk(1, dim=-1)
#             x = topi.squeeze(-1).detach()
            tokens = x.squeeze(1).cpu().numpy()
            output.append(tokens)
    
    output = np.array(output).T
    for s in output:
        out = idx2sentence(s, idx2rus)
        print(out.replace('PAD', ""))

    
evaluate("What is going on?")

In [0]:
Варианты простых улучшений:
1. Attention over encoder outputs (try decomposable attention)
2. seq2seq + Autoencoder, с возможностью перевода lang->state->lang
3. BPE